In [1]:
# The script version of this tool can be found at: 
#      repo/emulator_Nx2pt/emu_Nx2pt/gen_train_set/draw_pco_params.py

In [2]:
import numpy as np
from pyDOE import lhs
import time

In [3]:
par_lim = {
    'Omega_m': [ 0.2  , 0.4   ],
    'sigma_8': [ 0.8  , 0.85  ],
    'Omega_b': [ 0.042, 0.050 ],
    'n_s'    : [ 0.9  , 1.02  ], 
    'h'      : [ 0.6  , 0.8   ],
}
# 1,000,000 points
# avg. x^2 (exclude boundary) ... 
# 

In [4]:
# par_keys = ['Omega_m', 'sigma_8', 'Omega_b', 'n_s', 'h']

# domain = np.zeros((2, len(par_keys)))
# for i, key in enumerate(par_keys):
#     domain[0][i] = par_lim[key][0]
#     domain[1][i] = par_lim[key][1]
# domain

In [5]:
def draw_lhs_samples(Npts, par_lim, criterion='cm', par_keys=None):
    '''
    criterion: a string that tells lhs how to sample the points (default: None, which simply randomizes the points within the intervals):
        “center” or “c”: center the points within the sampling intervals
        “maximin” or “m”: maximize the minimum distance between points, but place the point in a randomized location within its interval
        “centermaximin” or “cm”: same as “maximin”, but centered within the intervals
        “correlation” or “corr”: minimize the maximum correlation coefficient
    '''
    
    if not par_keys:
        par_keys = par_lim.keys()
    
    Ndim = len(par_lim)
    
    domain = np.zeros((2, Ndim))
    for i, key in enumerate(par_keys):
        domain[0][i] = par_lim[key][0]
        domain[1][i] = par_lim[key][1]
    
    #start = time.time()
    samples = lhs(Ndim, samples=Npts, criterion=criterion)
    #end = time.time()
    #period = (end - start)/60.
    #print(f'lhs finished: {period} mins')
    
    samples = domain[0] + samples * (domain[1] - domain[0])

    return samples

In [6]:
Npts = 100
pco_samples = draw_lhs_samples(Npts=Npts, par_lim=par_lim, par_keys=['Omega_m', 'sigma_8', 'Omega_b', 'n_s', 'h'])

In [7]:
pco_samples.shape

(100, 5)

In [8]:
data_dir = '/home/hhg/Research/emu_Nx2pt/data/'
filename = data_dir+f'pco_m8bnh_{Npts}.pkl'
print(f'output file to:', filename)

output file to: /home/hhg/Research/emu_Nx2pt/data/pco_m8bnh_100.pkl


In [9]:
import pickle

with open(filename, 'wb') as handle:
    pickle.dump(pco_samples, handle)

### Time Estimate of pyDOE.lhs

In [10]:
from pyDOE import lhs

In [11]:
%time samples_ori = lhs(5, samples=1000, criterion='cm')

CPU times: user 8.77 s, sys: 89.8 ms, total: 8.86 s
Wall time: 8.86 s


In [12]:
%time samples_ori = lhs(5, samples=2000, criterion='cm')

CPU times: user 35.5 s, sys: 169 ms, total: 35.7 s
Wall time: 35.7 s


In [13]:
%time samples_ori = lhs(5, samples=4000, criterion='cm')

CPU times: user 2min 22s, sys: 861 ms, total: 2min 23s
Wall time: 2min 23s


- It looks like the run time of `pyDOE.lhs` scales as O(n^2).
- If drawing 1000 points takes ~10 seconds. 0.6M points would take me ~ 600^2x10 seconds = 1000 hours...